In [ ]:
import tensorflow as tf
from keras_preprocessing import image

train_dir = '10_food_classes_10_percent/train'
test_dir = '10_food_classes_10_percent/test'
IMG_SIZE = (224,224)
BATCH_SIZE = 32
train_data_10_percent=tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                           image_size=IMG_SIZE,
                                                                          label_mode="categorical",
                                                                          batch_size=BATCH_SIZE)
test_data=tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                              image_size=IMG_SIZE,
                                                              label_mode="categorical",
                                                              batch_size=BATCH_SIZE)

### MODEL 0 transfer learning model using functional api


In [ ]:
from keras.api.applications import EfficientNetB0
from keras.api.layers import Flatten, Dense, GlobalAveragePooling2D, Dropout, Input
from keras.api.models import Model
from TENSORFLOW.helper_functions import create_tensorboard_callback, plot_loss_curves
from keras.api.optimizers import Adam
from keras.api.losses import categorical_crossentropy

#create base model
base_model = EfficientNetB0(include_top=False)

#freeze base model
base_model.trainable = False

#create input into our model
inputs = Input(shape=(224, 224, 3), name="input_layer")

#pass the input to the base model
x = base_model(inputs)
print(f"Shape after passing input through the base model: {x.shape}")

x = GlobalAveragePooling2D(name="global_average_pooling_layer")(x)

print(f"Shape after global average pooling layer: {x.shape}")

outputs = Dense(10, activation="softmax", name="output_layer")(x)

model_0 = Model(inputs=inputs, outputs=outputs)

model_0.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=["accuracy"])

history_10 = model_0.fit(train_data_10_percent, epochs=5,
                         steps_per_epoch=len(train_data_10_percent),
                         validation_data=test_data,
                         validation_steps=int(0.25 * len(test_data)),
                         callbacks=[create_tensorboard_callback(dir_name="transfer_learning",
                                                                experiment_name="10%_feature_extraction")], verbose=2)

In [ ]:
model_0.evaluate(test_data)

In [ ]:
for layer_number, layer in enumerate(base_model.layers):
    print(layer_number,layer.name)

In [ ]:
model_0.summary()

In [ ]:
from TENSORFLOW.helper_functions import plot_loss_curves, pred_and_plot, compare_historys,confusion_matrix

plot_loss_curves(history_10)

In [ ]:
import zipfile
!curl -O https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_1_percent.zip

zip_ref=zipfile.ZipFile('10_food_classes_1_percent.zip')
zip_ref.extractall()
zip_ref.close()

train_dir_1_percent = "10_food_classes_1_percent/train/"
test_dir = "10_food_classes_1_percent/test/"

In [ ]:
import tensorflow as tf
IMG_SIZE = (224, 224)
train_data_1_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir_1_percent,
                                                                           label_mode="categorical",
                                                                           batch_size=32, # default
                                                                           image_size=IMG_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Data augmentation pipeline
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomHeight(0.2),
    layers.RandomWidth(0.2),
    # layers.Rescaling(1./255)  # Uncomment for ResNet50V2, remove for EfficientNetV2B0
], name="data_augmentation")

print("Data augmentation layer created successfully!")


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import random

target_class = random.choice(train_data_1_percent.class_names)  # choose a random class
target_dir = "10_food_classes_1_percent/train/" + target_class  # create the target directory
random_image = random.choice(os.listdir(target_dir))  # choose a random image from target directory
random_image_path = target_dir + "/" + random_image  # create the choosen random image path
img = mpimg.imread(random_image_path)  # read in the chosen target image
plt.imshow(img)  # plot the target image
plt.title(f"Original random image from class: {target_class}")
plt.axis(False);  # turn off the axes

# Augment the image
augmented_img = data_augmentation(tf.expand_dims(img, axis=0))
# data augmentation model requires shape (None, height, width, 3)

plt.figure()
plt.imshow(tf.squeeze(augmented_img) / 255.)  # requires normalization after augmentation
plt.title(f"Augmented random image from class: {target_class}")
plt.axis(False);